In [2]:
import numpy as np
import datasets

datasets.downloads = '/home/greg/Downloads/'

In [3]:
trX, teX, trY, teY = datasets.mnist()

In [4]:
epochs = 1
batch_size = 32
learning_rate = 0.1
H = 100
# maybe preprocess X
R = lambda Ws: 0 # regularization penalty
init_weights = lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * 0.05
# init_weights - try glorot
accuracy = lambda sm, y: np.mean(np.argmax(sm, axis = 1) == np.argmax(y, axis = 1))

trY = datasets.one_hot(trY)
teY = datasets.one_hot(teY)

N, D = trX.shape
N, C = trY.shape

In [31]:
from keras.models import Model
from keras.layers import Input, Dense

X = Input(shape = (D,))
H1 = Dense(H, activation = 'relu')(X)
H2 = Dense(C, activation = 'softmax')(H1)
model = Model(input = X, output = H2)

In [33]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(H, input_dim = D, activation = 'relu'))
model.add(Dense(C, activation = 'softmax'))

In [32]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = epochs)
sm = model.predict(teX)
print accuracy(sm, teY)

Epoch 1/1
60000/60000 [==============================] - 9s - loss: 0.2760     
0.9578


In [ ]:
config = model.get_config()
weights = model.get_weights()

model = Sequential.from_config(config)
model.set_weights(weights)

In [13]:
import tensorflow as tf

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  

x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
H1 = tf.nn.relu(dense(H, x))  
H2 = dense(trY.shape[1], H1)
loss = tf.nn.softmax_cross_entropy_with_logits(H2, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for _ in xrange(epochs):
        for start in xrange(0, len(trX) - batch_size + 1, batch_size):
            end = start + batch_size            
            feed_dict = {x: trX[start : end], y: trY[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    sm = H2.eval({x: teX})
    print accuracy(sm, teY)

0.9476


In [ ]:
# https://github.com/Newmu/Theano-Tutorials

import theano
from theano import tensor as T

x = T.fmatrix()
y = T.fmatrix()

W1 = theano.shared(init_weights(D, H))
b1 = theano.shared(np.zeros(H) * 1.0)
W2 = theano.shcared(init_weights(H, C))
b2 = theano.shared(np.zeros(C) * 1.0)

H1 = T.nnet.relu(T.dot(x, W1) + b1)
H2 = T.nnet.softmax(T.dot(H1, W2) + b2)

loss = T.mean(T.nnet.categorical_crossentropy(H2, y))
updates = [[p, p - T.grad(loss, p) * learning_rate] for p in [W1, W2]] # b1, b2?
optimizer = theano.function([x, y], updates = updates, allow_input_downcast = True)

for _ in xrange(epochs):
    for start in xrange(0, len(trX) - batch_size + 1, batch_size):
        end = start + batch_size
        optimizer(trX[start : end], trY[start : end])
        
W1, b1 = W1.get_value(), b1.get_value()
W2, b2 = W2.get_value(), b2.get_value()
sm = softmax(np.maximum(0, teX.dot(W1) + b1).dot(W2) + b2)
print accuracy(sm, teY)

if False:
    # you can play with theano to write derivative functions

    x = T.fmatrix()
    y = T.fmatrix()
    loss = T.mean(T.nnet.categorical_crossentropy(T.nnet.softmax(x), y))
    grad_loss = theano.function([x, y], T.grad(loss, x))

    x = [[-3, 3], [-2, 1], [-2, 1]]
    y = [[0.0, 1], [0, 1], [1, 0]]
    grad_loss(x, y)

In [ ]:
from sklearn.neural_network import MLPClassifier

X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state = 1)
clf.fit(X, y)
"""
MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
"""
clf.predict([[2., 2.], [-1., -2.]])

In [15]:
import numpy as np
from softmax import softmax

W1 = init_weights(D, H)
b1 = np.zeros(H)
W2 = init_weights(H, C)
b2 = np.zeros(C)

for _ in xrange(epochs):
    for start in xrange(0, len(trX) - batch_size + 1, batch_size):
        end = start + batch_size
        X = trX[start : end]
        Y = trY[start : end]

        # feed forward
        H1 = np.maximum(0, X.dot(W1) + b1)
        H2 = softmax(H1.dot(W2) + b2)
        
        # loss = -np.mean(np.log(np.sum(H2 * Y, axis = 1)))

        # back propagate
        grad_H2 = (H2 - Y) / len(Y)
        grad_W2 = H1.T.dot(grad_H2)
        grad_b2 = np.sum(grad_H2, axis = 0)

        grad_H1 = np.zeros_like(H1)
        grad_H1[H1 > 0] = 1
        grad_H1 *= grad_H2.dot(W2.T)
        grad_W1 = X.T.dot(grad_H1)
        grad_b1 = np.sum(grad_H1, axis = 0)

        # update
        W2 -= grad_W2 * learning_rate
        b2 -= grad_b2 * learning_rate
        W1 -= grad_W1 * learning_rate
        b1 -= grad_b1 * learning_rate

sm = softmax(np.maximum(0, teX.dot(W1) + b1).dot(W2) + b2)
print accuracy(sm, teY)

0.9375


In [437]:
# http://localhost:8888/edit/Desktop/assignment1/cs231n/gradient_check.py

def numerical_gradient(f, x):
    h = 0.00001
    grad = np.zeros_like(x)
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h
        fxph = f(x)
        x[ix] = oldval - h
        fxmh = f(x)
        x[ix] = oldval
        grad[ix] = (fxph - fxmh) / (2 * h)
        it.iternext()
        
    return grad

x = np.array([1, 2])
f = lambda x: np.sum(x)
numerical_gradient(f, x)

array([49999, 49999])